In [ ]:
# Importing Required Modules 
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Set the random seeds. Do not change this!
seedVal = 41
random.seed(seedVal)
np.random.seed(seedVal)

In [ ]:
!git clone https://github.com/Revanth980727/Computational-Physics-PHYS-5070.git

Cloning into 'Computational-Physics-PHYS-5070'...
remote: Enumerating objects: 7551, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 7551 (delta 18), reused 48 (delta 14), pack-reused 7499
Receiving objects: 100% (7551/7551), 96.14 MiB | 17.15 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [ ]:
file_names = list()
true_label = list()
with open("/content/Computational-Physics-PHYS-5070/labels-map-proj-v3.txt",'r') as data_file:
    for line in data_file:
        data = line.split()
        file_names.append(data[0])
        true_label.append(data[1])
print(len(file_names))

73031


In [ ]:
from pathlib import Path
dataset = list()
labels = list()
for i in range(0,len(file_names)):
    my_file = Path('/content/Computational-Physics-PHYS-5070/map-proj-v3/'+file_names[i])
    #print(my_file)
    if my_file.is_file():
        img = cv2.imread('/content/Computational-Physics-PHYS-5070/map-proj-v3/'+ file_names[i],0)
        dataset.append(img)
        labels.append(int(true_label[i]))
#print(counter)
print(len(dataset))
print(len(labels))

7495
7495


In [ ]:
train_x = dataset[0:6000]
print(len(train_x))
train_x = np.array(train_x)
train_y = labels[0:6000]
print(len(train_y))
train_y = np.array(train_y)
test_x = dataset[6000:]
print(len(test_x))
test_x = np.array(test_x)
test_y = labels[6000:]
test_set = test_x
print(len(test_y))
test_y = np.array(test_y)

6000
6000
1495
1495


In [ ]:
dataset = np.array(dataset)
labels = np.array(labels)

In [ ]:
import pandas as pd
df = pd.read_csv (r'/content/Computational-Physics-PHYS-5070/landmarks_map-proj-v3_classmap.csv',header = None, names = ['Label','Class'])
#print(df)
#print(df['Class'])
df.head()

,Label,Class
0,0,other
1,1,crater
2,2,dark dune
3,3,slope streak
4,4,bright dune


In [ ]:
import keras
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# Define some constants.
NUM_CLASSES = 8
BATCH_SIZE = 32

In [ ]:
# Normalizing the dataset
train_x = keras.utils.all_utils.normalize(train_x, axis=1)
test_x = keras.utils.all_utils.normalize(test_x, axis=1)

# Class vectors are converted to binary class matrices
train_y = keras.utils.all_utils.to_categorical(train_y, NUM_CLASSES)
test_y =keras.utils.all_utils.to_categorical(test_y, NUM_CLASSES)
print(len(train_y[0]))

K.set_image_data_format('channels_first')

# Data reshape
train_x = train_x.reshape(train_x.shape[0], 1, 227, 227)
test_x =  test_x.reshape(test_x.shape[0], 1, 227, 227)

8


In [ ]:
#Defining ANN model
nasa_model = Sequential()

nasa_model.add(Flatten(input_shape=(1,227,227)))
nasa_model.add(Dense(256,activation='relu'))
nasa_model.add(Dropout(0.5))
nasa_model.add(Dense(128,activation='relu'))
nasa_model.add(Dropout(0.5))


nasa_model.add(Dense(NUM_CLASSES, activation='softmax'))

In [ ]:
nasa_model.compile(loss=keras.losses.categorical_crossentropy,optimizer='adam',metrics=['accuracy'])

In [ ]:
nasa_model.fit(train_x, train_y, epochs=18, batch_size=BATCH_SIZE)

Train on 6000 samples
Epoch 1/18
6000/6000 [==============================] - 4s 606us/sample - loss: 0.6125 - accuracy: 0.8603
Epoch 2/18
6000/6000 [==============================] - 4s 595us/sample - loss: 0.6135 - accuracy: 0.8603
Epoch 3/18
6000/6000 [==============================] - 4s 595us/sample - loss: 0.6112 - accuracy: 0.8603
Epoch 4/18
6000/6000 [==============================] - 4s 613us/sample - loss: 0.6121 - accuracy: 0.8603
Epoch 5/18
6000/6000 [==============================] - 4s 615us/sample - loss: 0.6120 - accuracy: 0.8603
Epoch 6/18
6000/6000 [==============================] - 4s 610us/sample - loss: 0.6118 - accuracy: 0.8603
Epoch 7/18
6000/6000 [==============================] - 4s 621us/sample - loss: 0.6119 - accuracy: 0.8603
Epoch 8/18
6000/6000 [==============================] - 4s 659us/sample - loss: 0.6129 - accuracy: 0.8603
Epoch 9/18
6000/6000 [==============================] - 4s 640us/sample - loss: 0.6117 - accuracy: 0.8603
Epoch 10/18
6000/6000 [=

In [ ]:
# Testing model on test dataset

predictions = nasa_model.predict(test_x)
count = 0
freq = dict()
incorrect = dict()
survey = dict()
for i in range(test_x.shape[0]):
  if np.where(predictions[i] == max(predictions[i]))[0][0] == np.where(test_y[i] == max(test_y[i]))[0][0]:
    class_type = np.where(predictions[i] == max(predictions[i]))[0][0]
    if class_type not in freq:
      freq[class_type] = i
      survey[class_type] = 1
    survey[class_type] += 1
    count+=1
  else:
    class_type = np.where(test_y[i] == max(test_y[i]))[0][0]
    if class_type not in incorrect:
      incorrect[class_type] = i
      survey[class_type] = 0
accuracy = count/test_x.shape[0]
print('Accuracy of the model = {}'.format(accuracy))

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Accuracy of the model = 0.7297658862876254
